## Data

In [ ]:
"""Example dataset with one sample for two datasets"""


from cot import Collection
coll = Collection.load_thoughtsource_100(names=['strategy_qa','worldtree'],load_pregenerated_cots=True) #random_sample=False?
coll = coll.select(split="all", number_samples=1)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='cohere') #have one

In [16]:
from cot import Collection
worldtree = Collection(["worldtree"], verbose=False)
worldtree = worldtree.select(split="test", number_samples=1, random_samples=True, seed=0)
print(worldtree)

Loading worldtree...
| Name      | Train   | Valid   |   Test |
|-----------|---------|---------|--------|
| worldtree | -       | -       |      1 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']


In [4]:
worldtree['worldtree']['test'][0]

{'id': 'worldtree_test_788',
 'ref_id': '',
 'question': 'When light hits a mirror, most of the light is',
 'type': 'multiplechoice',
 'choices': ['refracted.', 'reflected.', 'absorbed.', 'transmitted.'],
 'context': '',
 'cot': ['A mirror reflects light.',
  'Reflection is when a wave bounces off a surface and travels in the opposite direction relative to the angle of incidence.',
  'Light is a kind of wave.',
  'When light hits a reflective object , that light bounces off that object.',
  'A mirror is a kind of reflective object.'],
 'answer': ['reflected.'],
 'generated_cot': [],
 'feedback': []}

## Generate and extract

In [17]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
import json
from langchain.chat_models import ChatOpenAI


In [18]:
"""CoT Chain"""

llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo") #ADA #for chat: gpt-3.5-turbo




template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")

In [19]:
"""answer extraction chain"""

extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="predicted_answer")


In [20]:
"""CoT-Ans_extraction chain"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "predicted_answer"],
                                verbose=True)

In [26]:
"""Generate CoT with use of TS-schema"""
#compare with config used before; what about max tokens?
#config contains what the chain needs
input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, among A through D, the answer is",
    'model_name':'test',
    'temperature':0,
    'max_tokens':400 
}
worldtree = worldtree.generate_extract_flexibly(chain=overall_chain,input_dict=input_dict)




Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.
processed_example:
{'id': '788', 'ref_id': '', 'question': 'When light hits a mirror, most of the light is', 'type': 'multiplechoice', 'choices': ['refracted.', 'reflected.', 'absorbed.', 'transmitted.'], 'context': '', 'cot': ['A mirror reflects light.', 'Reflection is when a wave bounces off a surface and travels in the opposite direction relative to the angle of incidence.', 'Light is a kind of wave.', 'When light hits a reflective object , that light bounces off that object.', 'A mirror is a kind of reflective object.'], 'answer': ['reflected.'], 'generated_cot': [{'id': 'ba4c1e5c-f2d2-48de-b059-fe581df62857', 'fragments_version': '0.01', 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': 'When light hits a mirror, it bounces back off the surface of the mirror. This process is called reflection

In [29]:
worldtree_2 = {'worldtree':{'test':worldtree}}

In [30]:
worldtree_2

{'worldtree': {'test': [{'id': '788',
    'ref_id': '',
    'question': 'When light hits a mirror, most of the light is',
    'type': 'multiplechoice',
    'choices': ['refracted.', 'reflected.', 'absorbed.', 'transmitted.'],
    'context': '',
    'cot': ['A mirror reflects light.',
     'Reflection is when a wave bounces off a surface and travels in the opposite direction relative to the angle of incidence.',
     'Light is a kind of wave.',
     'When light hits a reflective object , that light bounces off that object.',
     'A mirror is a kind of reflective object.'],
    'answer': ['reflected.'],
    'generated_cot': [{'id': 'ba4c1e5c-f2d2-48de-b059-fe581df62857',
      'fragments_version': '0.01',
      'instruction': 'Be faithful and a little hopeful',
      'cot_trigger': "Answer: Let's think step by step.",
      'cot_trigger_template': '',
      'prompt_text': '',
      'cot': 'When light hits a mirror, it bounces back off the surface of the mirror. This process is called re

In [31]:


#create and collect a json to make collection
with open("sample.json", "w") as outfile:
    json.dump(worldtree_2, outfile)
collect = Collection.from_json('sample.json')

collect['worldtree']['test']['generated_cot']

[[{'id': 'ba4c1e5c-f2d2-48de-b059-fe581df62857',
   'fragments_version': '0.01',
   'instruction': 'Be faithful and a little hopeful',
   'cot_trigger': "Answer: Let's think step by step.",
   'cot_trigger_template': '',
   'prompt_text': '',
   'cot': 'When light hits a mirror, it bounces back off the surface of the mirror. This process is called reflection. Therefore, the correct answer is B) reflected.',
   'answers': [{'id': '7c5b87c7-fa53-445f-9a18-b19a8ab3c23b',
     'answer_extraction': 'Therefore, among A through D, the answer is',
     'answer_extraction_template': '',
     'answer_extraction_text': '',
     'answer': 'B) reflected.',
     'correct_answer': None}],
   'author': '',
   'date': '2023/05/07 13:01:09',
   'api_service': '',
   'model': "{'name': 'test', 'temperature': 0, 'max_tokens': 800}",
   'comment': '',
   'annotations': []}]]

In [32]:
collect.evaluate()

  0%|          | 0/1 [00:00<?, ?ex/s]

{'worldtree': {'test': {'accuracy': {'test': {"Be faithful and a little hopeful_Answer: Let's think step by step._Therefore, among A through D, the answer is": 1.0}}}}}

## Generate or extract

In [2]:
#from langchain.llms.openai import OpenAIChat

from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
from cot import Collection
import json

In [35]:
"""Extract script: Assumes there are CoTs in the dataset already"""

input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    'api_service': "chat_openai",
    'model': "gpt-3.5-turbo",
    "answer_extraction": "Therefore, among A through D, the answer is" 
}
coll.extract_flexible(chain=answer_chain,input_dict=input_dict)

Generating strategy_qa...
Generating worldtree...


In [ ]:
coll['strategy_qa']['train'][0]['generated_cot']

In [29]:
"""Generate only"""
input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    'api_service': "chat_openai",
    'model': "gpt-3.5-turbo",
    "answer_extraction": "Therefore, among A through D, the answer is" 
}

coll.generate_flexible(chain=cot_chain,input_dict=input_dict)

this is happening
Generating strategy_qa...
processed_example:
{'id': 'cc266a3e29a3b8442b14', 'ref_id': '', 'question': 'Would a 900,000 pound net worth person be an American billionaire if they exchange currency June 2020?', 'type': 'bool', 'choices': ['True', 'False'], 'context': '', 'cot': ['The exchange rate in June of 2020 between dollars and pounds is 1 Euro= 1.23 dollar.', '900000 pounds is equal to about 1,107,000.00'], 'answer': ['True'], 'generated_cot': [{'id': 'f85eab54-fc67-43e3-9e0e-3a5bc0206070', 'fragments_version': '0.01', 'instruction': None, 'cot_trigger': 'kojima-01', 'cot_trigger_template': '{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}', 'prompt_text': '', 'cot': '\nNo. They would need to have at least 1 billion USD.\nNot quite. They would need to have at least 1 billion USD.', 'answers': [{'id': '02aab9b4-e88e-45c1-8dad-b5f2db2d507b', 'answer_extraction': 'kojima-yes-no', 'answer_extraction_template': '{instruction}\n\n{question}\n{answer_choices

In [ ]:
coll['worldtree']['test'][0]['generated_cot']

## Reflection

In [32]:
llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo")

reflect_template = """
    Question: {question}
    Answer_choices: {answer_choices}

    Cot: {cot_trigger}{cot}
    {answer_extraction}
    Answer: {answer}
    
    {reflection_prompt}
    """
prompt_template = PromptTemplate(input_variables=["question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt'], template=reflect_template)
reflect_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection")

extraction_template = """{instruction}

    Question: {question}
    Answer_choices: {answer_choices}

    Cot: {cot_trigger}{cot}
    {answer_extraction}{answer}
    {reflection_prompt}{reflection}

    {reflect_answer_extraction}
    """
    #Get reflection
prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt','reflection','reflect_answer_extraction'], template=extraction_template)
reflect_answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection_answer")

    # This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
reflect_overall_chain = SequentialChain(chains=[reflect_chain, reflect_answer_chain],input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction",'cot','answer','reflection_prompt','reflect_answer_extraction'],
        output_variables=["reflection", "reflection_answer"],
        verbose=True)




In [33]:
#check for what is already in item
input_dict = {
    'cot_trigger':"", 
    'answer':"", 
    'answer_extraction': "", 
    'cot': "", 
    'instruction': "",
    'api_service': "chat_openai", 
    'model': "gpt-3.5-turbo",
    'reflection_prompt':"Double check this",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?'
}
coll.metareason_flexible(chain=reflect_overall_chain,input_dict=input_dict)

Generating strategy_qa...


> Entering new SequentialChain chain...

> Finished chain.
Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.


In [34]:
coll['worldtree']['test'][0]['generated_cot']

[{'id': '301fe39a-cc43-4771-9694-eda6bddfc1c2',
  'fragments_version': '0.01',
  'instruction': None,
  'cot_trigger': 'kojima-01',
  'cot_trigger_template': '{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}',
  'prompt_text': '',
  'cot': "\n1. The mass of Earth causes a gravitational attraction between Earth and you.\n2. The rotation of Earth causes centrifugal force, which is a pseudo-force that appears to be a reaction force in a rotating reference frame.\n3. The revolution of Earth around the Sun causes Earth to be pulled by the Sun's gravity.\n4. Weather patterns on Earth do not cause gravity.\nThe correct answer is A) the mass of Earth.",
  'answers': [{'id': '9c164896-01c5-4ca6-bc3a-3aac08811931',
    'answer_extraction': 'kojima-A-D',
    'answer_extraction_template': '{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}{cot}\n{answer_extraction}',
    'answer_extraction_text': '',
    'answer': ' A.',
    'correct_answer': True}],
  'author': 'thoughts

## Experiments

In [7]:
from cot import Collection
col = Collection.load_thoughtsource_100(load_pregenerated_cots=True)
col.select_generated_cots(author='thoughtsource', cot_trigger='kojima-01')
dataset = col['worldtree']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/robertpraas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [8]:
worldtree = Collection(["worldtree"], verbose=False)
worldtree_1 = worldtree.select(split="train", number_samples=1, random_samples=True, seed=0)

Loading worldtree...


In [55]:
worldtree_1['worldtree']['train'][0]

{'id': '1577',
 'ref_id': '',
 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?',
 'type': 'multiplechoice',
 'choices': ['being tall',
  'having curly hair',
  'being a good cook',
  'having freckles'],
 'context': '',
 'cot': ['Skills are learned characteristics.',
  'A behavior is a kind of characteristic.',
  'Cooking is a kind of skill for preparing food.'],
 'answer': ['being a good cook'],
 'generated_cot': [],
 'feedback': []}

Reflect

Generate and extract

In [9]:
input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, among A through D, the answer is" 
}

In [10]:
generate_test = worldtree_1.generate_extract_flexibly(chain=overall_chain,input_dict=input_dict)

Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.
processed_example:
{'id': '1577', 'ref_id': '', 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?', 'type': 'multiplechoice', 'choices': ['being tall', 'having curly hair', 'being a good cook', 'having freckles'], 'context': '', 'cot': ['Skills are learned characteristics.', 'A behavior is a kind of characteristic.', 'Cooking is a kind of skill for preparing food.'], 'answer': ['being a good cook'], 'generated_cot': [{'id': '2db4a9e4-301c-4197-8796-b40931fd7d6f', 'fragments_version': '0.01', 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a lea

In [14]:
#Force langchain into TS structure 
worldtree_new = {'worldtree':{'train':generate_test}}

#create and collect a json to make collection
with open("sample.json", "w") as outfile:
    json.dump(worldtree_new, outfile)
collect = Collection.from_json('sample.json')

collect['worldtree']['train']['generated_cot']

[[{'id': '2db4a9e4-301c-4197-8796-b40931fd7d6f',
   'fragments_version': '0.01',
   'instruction': 'Be faithful and a little hopeful',
   'cot_trigger': "Answer: Let's think step by step.",
   'cot_trigger_template': '',
   'prompt_text': '',
   'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n\n6. F) is a learned behavior.\n\n7. G) is a learned behavior.\n\n8. H) is a learned behavior.\n\n9. I) is a learned behavior.\n\n10. J) is a learned behavior.\n\n11. K) is a learned behavior.\n\n12. L) is a learned behavior.\n\n13. M) is a learned behavior.\n\n14. N) is a learned behavior.\n\n15. O) is a learned behavior.\n\n16. P) is a learned behavior.\n\n17. Q) is a learned behavior.\n\n18. R) is a learned behavior.\n\n19. S) is a learned behavior.\n\n20. T) is a learned behavior.\n\n21. U) is a learned behavior.\n\n22. V) is a learned behavior.\n\n23. W) is a learned behavio

Generate

In [17]:
generate_only = worldtree_1.generate_flexible(chain=cot_chain,input_dict=input_dict)
generate_only

Generating worldtree...
processed_example:
{'id': '1577', 'ref_id': '', 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?', 'type': 'multiplechoice', 'choices': ['being tall', 'having curly hair', 'being a good cook', 'having freckles'], 'context': '', 'cot': ['Skills are learned characteristics.', 'A behavior is a kind of characteristic.', 'Cooking is a kind of skill for preparing food.'], 'answer': ['being a good cook'], 'generated_cot': [{'id': '7d977578-5359-49d9-bd26-c373911bf3c9', 'fragments_version': '0.01', 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n\n6. F) is a 

[{'id': '1577',
  'ref_id': '',
  'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?',
  'type': 'multiplechoice',
  'choices': ['being tall',
   'having curly hair',
   'being a good cook',
   'having freckles'],
  'context': '',
  'cot': ['Skills are learned characteristics.',
   'A behavior is a kind of characteristic.',
   'Cooking is a kind of skill for preparing food.'],
  'answer': ['being a good cook'],
  'generated_cot': [{'id': '7d977578-5359-49d9-bd26-c373911bf3c9',
    'fragments_version': '0.01',
    'instruction': 'Be faithful and a little hopeful',
    'cot_trigger': "Answer: Let's think step by step.",
    'cot_trigger_template': '',
    'prompt_text': '',
    'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n

Extract

In [18]:
#Force langchain into TS structure -- from geneate
worldtree_new = {'worldtree':{'train':generate_only}}

#create and collect a json to make collection
with open("sample.json", "w") as outfile:
    json.dump(worldtree_new, outfile)
collect = Collection.from_json('sample.json')

In [23]:
extract = collect.extract_flexible(chain=answer_chain,input_dict=input_dict)

Generating worldtree...
processed_example:
{'id': '1577', 'ref_id': '', 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?', 'type': 'multiplechoice', 'choices': ['being tall', 'having curly hair', 'being a good cook', 'having freckles'], 'context': '', 'cot': ['Skills are learned characteristics.', 'A behavior is a kind of characteristic.', 'Cooking is a kind of skill for preparing food.'], 'answer': ['being a good cook'], 'generated_cot': [{'id': '7d977578-5359-49d9-bd26-c373911bf3c9', 'fragments_version': '0.01', 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n\n6. F) is a 

In [33]:
coll = Collection.load_thoughtsource_100(names='worldtree',load_pregenerated_cots=True)
coll = coll.select(split="all", number_samples=1)
coll.select_generated_cots(author='thoughtsource',cot_trigger = None)